# POLYGON

## Enunciado

Implementa un procedimiento para mejorar la aproximación poligonal obtenida por el método `cv.approxPolyDP` permitiendo vértices fuera del contorno de entrada. Da una medida de la calidad de la aproximación. Pruébalo construyendo un cuadrilátero a partir de la silueta de un carnet o una tarjeta con las esquinas redondeadas.

## Explicación general del procedimiento implementado

Para mejorar la aproximación de un contorno, permitiendo vértices fuera del contorno de entrada, se han realizado los siguientes pasos:

1. Se ha ejecutado **cv.approxPolyDP**, de forma que se obtiene un contorno más simplificado.
2. Como este método no funciona para esquinas redondeadas, se ha incluido un *factor de calidad* con un valor perteneciente a \[0,1), de forma que **se eliminan las rectas** del contorno cuya **longitud sea demasiado pequeña** comparada con el resto (factor$*$longitud<longitud de la mayor línea).
3. Una vez eliminadas las líneas demasiado pequeñas, para obtener las esquinas redondeadas, se calcula la **intersección entre las rectas** que se encontraban **conectadas a las rectas pequeñas que se han eliminado**, y se añade el punto de intersección al contorno, de forma que se obtiene la esquina que estas realizan.
4. Una vez obtenido todo esto, se vuelve a realizar **cv.approxPolyDP**, de forma que se termina de simplificar el contorno.

De esta forma, las rectas con un tamaño menor a un tanto por ciento de la recta más larga (el tanto por ciento se especifica por el *factor de calidad*), se consideran pertenecientes al redondeo de alguna esquina, por lo que se eliminan y se calcula la esquina correspondiente.



## Código

In [ ]:
from umucv.stream   import autoStream
import cv2 as cv
import numpy as np
from umucv.contours import extractContours
import matplotlib.pyplot as plt

En primer lugar, se realiza un método para mejorar la aproximación del contorno pasado como parámetro (los pasos 2 y 3 de la explicación general). En vez de explicarlo en un fragmento de código, se explica en comentarios del código porque se considera que así se entiende mejor.

In [ ]:
# Mejora la aproximación del contorno pasado como parámetro
def mejorar_aproximacion(contorno, factor_precision=0.25):
    # copia de contorno
    contorno_parametro = np.array(contorno)
    # pone el contorno, en formato (n, 2) siendo n el número de puntos del contorno
    contorno = contorno.reshape(contorno.shape[0],2)

    # max_dist = Máxima distancia entre puntos de la silueta
    max_dist = 0.0
    for i in range(len(contorno)):
        max_dist = max(max_dist, distancia(contorno[i], contorno[i-1]))
          
    # puntos = Contorno con los puntos que pertenecen a rectas largas, y None para indicar que se añada una línea como intersección de las dos líneas largas
    # que se queden a los lados de None (para redondear las esquinas)
    puntos = []
    j = 0
    for i in range(1, len(contorno)-1, 1):
        # Si el punto no está en ninguna línea "larga"
        if ((distancia(contorno[i], contorno[i-1]) < factor_precision*max_dist) and (distancia(contorno[i], contorno[i+1]) < factor_precision*max_dist)):
            # Añadir None para alargar las rectas de los lados
            if j == 0 or puntos[j-1] is not None:
                puntos += [None]
                j += 1
        # Si el punto pertenece a una línea "larga" por la izquierda pero no por la derecha
        elif (distancia(contorno[i], contorno[i+1]) < factor_precision*max_dist):
            puntos += [contorno[i]]
            puntos += [None]
            j += 2
        # Si el punto pertenece a una línea larga por ambos lados
        else:
            puntos += [contorno[i]]
            j += 1
    
    # El mismo concepto pero con los puntos de las esquinas
    if (len(contorno) >= 2):
        # Punto [0]: No pertenece a ninguna recta "larga"
        if (distancia(contorno[0], contorno[1]) < factor_precision*max_dist) and (distancia(contorno[0], contorno[len(contorno)-1]) < factor_precision*max_dist):
            if j == 0 or puntos[0] is not None:
                puntos = [None] + puntos
                j += 1
        # Punto[0]: Pertenece a una recta "larga" por la izquierda pero no por la derecha
        elif (distancia(contorno[0], contorno[1]) < factor_precision*max_dist):
            if j == 0 or puntos[0] is not None:
                puntos = [contorno[0]] + [None] + puntos
                j += 2
            else:
                puntos = [contorno[0]] + puntos
                j += 1
        # Punto[0]: Pertenece a una línea larga por ambos lados
        else:
            puntos = [contorno[0]] + puntos
            j += 1
        # Punto[ultimo]: No pertenece a ninguna recta "larga"
        if (distancia(contorno[len(contorno)-2], contorno[len(contorno)-1]) < factor_precision*max_dist) and (distancia(contorno[0], contorno[len(contorno)-1]) < factor_precision*max_dist):
            if j == 0 or puntos[j-1] is not None:
                puntos += [None]
                j += 1
        # Punto[ultimo]: Pertenece a una recta "larga" por la izquierda pero no por la derecha
        elif (distancia(contorno[len(contorno)-1], contorno[0]) < factor_precision*max_dist):
            if j == 0 or puntos[0] is not None:
                puntos = puntos + [contorno[len(contorno)-1]] + [None]
                j += 2
            else:
                puntos = puntos + [contorno[len(contorno)-1]]
                j += 1
        # Punto[ultimo]: Pertenece a una línea larga por ambos lados
        else:
            puntos = puntos + [contorno[len(contorno)-1]]
            j += 1
        # Si se ha quedado con None al principio y al final, se elimina uno de ellos para evitar tener dos None seguidos
        if (puntos[0] is None and puntos[len(puntos)-1] is None):
            puntos = puntos[:-1]

    # Si la silueta tiene menos de 5 puntos, no hay esquinas a las que quitar el redondeo
    if len(puntos) < 5:
        return contorno_parametro
    
    # puntosFinal = Silueta con los puntos de "puntos" y en vez de None un punto que forma la intersección de las dos rectas que se quedan a los lados de None
    # Si no hay intersección entre las rectas (son paralelas), se pone el punto medio entre los puntos de los extremos del punto a añadir
    puntosFinal = np.array([])
    for i in range(len(puntos)):
        if puntos[i] is None:
            # Puntos que forman las rectas anterior y posterior
            r1_p1 = (i-1) % len(puntos)
            r1_p1 = puntos[r1_p1]
            r1_p2 = (i-2) % len(puntos)
            r1_p2 = puntos[r1_p2]
            r2_p1 = (i+1) % len(puntos)
            r2_p1 = puntos[r2_p1]
            r2_p2 = (i+2) % len(puntos)
            r2_p2 = puntos[r2_p2]
            # Rectas anterior y posterior
            recta1 = np.cross(np.array([r1_p1[0], r1_p1[1], 1]),np.array([r1_p2[0], r1_p2[1], 1]))
            recta2 = np.cross(np.array([r2_p1[0], r2_p1[1], 1]),np.array([r2_p2[0], r2_p2[1], 1]))
            # Intersección de rectas
            recta_h_interseccion = np.cross(recta1, recta2)
            if recta_h_interseccion[2] == 0: # Si las rectas son paralelas, el punto medio
                if len(puntosFinal) == 0: # Si es el primer punto
                    puntosFinal = np.array([[int((r1_p2[0]+r2_p1[0])/2),int((r1_p2[1]+r2_p1[1])/2)]])
                else:
                    puntosFinal = np.append(puntosFinal, [[int((r1_p2[0]+r2_p1[0])/2),int((r1_p2[1]+r2_p1[1])/2)]], axis=0)
            else: # Si las rectas no son paralelas, se añade el punto de intersección entre las rectas
                if len(puntosFinal) == 0: # Si es el primer punto
                    puntosFinal = np.array([[int(abs(recta_h_interseccion[0]/recta_h_interseccion[2])), int(abs(recta_h_interseccion[1]/recta_h_interseccion[2]))]])
                else:
                    puntosFinal = np.append(puntosFinal, [[int(abs(recta_h_interseccion[0]/recta_h_interseccion[2])), int(abs(recta_h_interseccion[1]/recta_h_interseccion[2]))]], axis=0)
        # Si el punto no es None, se añade a la silueta
        else:
            if len(puntosFinal) == 0: # Si es el primer punto
                puntosFinal = np.array([[puntos[i][0], puntos[i][1]]])
            else:
                puntosFinal = np.append(puntosFinal, [[puntos[i][0], puntos[i][1]]], axis=0)
    return puntosFinal

Posteriormente, se guarda la distancia entre 2 puntos en un método (*distancia*), ya que esto se necesita para la función anterior (*mejorar_aproximacion*).

In [ ]:
# Calcular distancia entre 2 puntos
def distancia(p1, p2):
    return np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

Se utiliza el método dado por el profesor en los [apuntes de la asignatura](https://github.com/albertoruiz/umucv/tree/master) para reducir los puntos de un contorno con la llamada a una función de OpenCV que lo realiza.

In [ ]:
# reducimos una polilínea con una tolerancia dada
def redu(c, eps=0.5):
    red = cv.approxPolyDP(c,eps,True)
    return red.reshape(-1,2)

A este método se pasa el contorno con los puntos, y como se dijo anteriormente, se reducen los puntos con la función de OpenCV, se mejora la aproximación de las esquinas permitiendo puntos fuera del contorno, y se vuelve a usar la función de OpenCV para volver a reducir los puntos.

In [ ]:
# filtramos una lista de contornos con aquellos
# que quedan reducidos a n vértices con la precisión deseada.
def polygons(cs,n,prec=1):
    # Quedarse con la silueta reducida
    rs = [ redu(c,prec) for c in cs ]
    # Mejora la aproximación de la silueta
    rs_mejorado = [mejorar_aproximacion(r, 0.25) for r in rs]
    # Vuelve a reducir la silueta
    rs_mejorado_reducido = [redu(r, prec) for r in rs_mejorado]
    # Quedarse con las siluetas que tengan n vértices
    resultado = [r for r in rs_mejorado_reducido if len(r) == n]
    return resultado

## Prueba con búsqueda de cuadriláteros en la imagen de un carnet

Se prueba la aproximación con la imagen de un carnet:

In [ ]:
# Se guarda la imagen en escala de grises para que tenga 1 solo valor y no 3 por píxel (como es el caso de BGR).
carnet = cv.imread('./carnet-falso.jpg')
# Se invierten los colores porque extractContours sólo extrae contornos de figuras oscuras sobre fondos claros
carnet = cv.bitwise_not(carnet)
carnet = cv.cvtColor(carnet,cv.COLOR_BGR2GRAY)
# Se extraen los contornos
cs = extractContours(carnet, minarea=10)
# Se llama al método que mejora la aproximación de los contornos (reduce los puntos) y se queda solamente con aquellos que tras la reducción tienen 4 vértices
good = polygons(cs, n=4, prec=3)
# Se vuelve a leer la imagen para que se vean los contornos sobre la imagen original
carnet = cv.imread('./carnet-falso.jpg')
# dibujamos en cyan todos los contornos interesantes
cv.drawContours(carnet,[c.astype(int) for c in cs], -1, (0,255,255), 2, cv.LINE_AA)
# dibujamos en rojo más grueso los cuadrados encontrados
cv.drawContours(carnet,[c.astype(int) for c in good], -1, (255,0,0), 3, cv.LINE_AA)
# Mostrar imagen con las líneas dibujadas
plt.figure(figsize=(8,6))
plt.imshow(carnet)
plt.show()

Como se puede observar, se ha eliminado el redondeo de las esquinas, con vértices para estas fuera del contorno de entrada.

## Teoría utilizada

Para obtener las intersecciones de rectas, se han utilizado las coordenadas homogéneas, cuya explicación se encuentra en [los apuntes de la asignatura](https://github.com/albertoruiz/umucv/blob/96a0e8bcc9e95151c309ac48f743e4fd8a90a77c/notebooks/coordhomog.ipynb)

A su vez, se ha realizado el código partiendo del programa [polygon0.py](https://github.com/albertoruiz/umucv/blob/96a0e8bcc9e95151c309ac48f743e4fd8a90a77c/code/polygon/polygon0.py).